# 🤖 Use RNN Models to Identify MPox

## 🗂️ Import Libraries

The following code block imports all the libraries that are necessary for our development purposes.

#### 🔢 `numpy`
- NumPy is a powerful Python library used for numerical computing, providing support for large, multi-dimensional arrays, matrices, and high-level mathematical functions to operate on them efficiently.

#### 🐼 `pandas`
- Pandas is a Python library designed for data manipulation and analysis, offering easy-to-use data structures like DataFrames and Series for handling structured data efficiently.

#### 👣 `pathlib`
- Pathlib is a Python library that provides an object-oriented interface for working with filesystem paths, making path manipulation and file operations more intuitive and cross-platform.

#### 📈 `matplotlib`
- Matplotlib is a Python library for creating static, interactive, and animated visualizations in a variety of formats, including plots, graphs, and charts.

#### ⚡ `tensorflow`
- TensorFlow is an open-source library designed for building and deploying machine learning and deep learning models, offering a flexible ecosystem for numerical computation and AI development.

#### 🧠 `keras`
- Keras is a high-level deep learning library in Python that simplifies the creation and training of neural networks by providing an intuitive interface to underlying frameworks like TensorFlow.

#### 🌊 `seaborn`
- Seaborn is a Python library built on Matplotlib that simplifies creating aesthetically pleasing and informative statistical graphics for data visualization.

#### 💻 `os`
- The os library in Python provides a way to interact with the operating system, enabling tasks such as file and directory manipulation, environment variable access, and process management.

#### 🔬 `scikit-learn`
- Scikit-learn is a Python library that provides simple and efficient tools for data mining, data analysis, and machine learning, including classification, regression, and clustering algorithms.

#### 📄 `shutil`
- The shutil library in Python provides a suite of functions for high-level file operations, including copying, moving, renaming, and deleting files and directories.

### 🎲 `random`
- The random library in Python provides functions to generate random numbers, select random elements, and perform random operations like shuffling and sampling, useful for simulations, games, and probabilistic programming.

### ⚠️ `warnings`
- The warnings library in Python is used to issue, control, and filter warning messages during runtime, helping developers identify and address potential issues in their code without interrupting program execution.

In [ ]:
## 🗂️ Import Libraries

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
import keras_tuner as kt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pathlib import Path
import random
import os

### Supress warning
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

## ♾️ Constants

While proceeding with the process, we need some constants that may refer to the directory paths, image category names, data generation properties, etc.

In [ ]:
## ♾️ Constants

### Directory paths
DATA_PATH = './data/original'
AUGMENTED_PATH = './data/rnn_augmented'
RNN_BALANCED_PATH = './data/rnn_balanced'
MODELS_PATH = './models/rnn'

TRAIN_DIRECTORY='train'
TEST_DIRECTORY='test'
VALIDATION_DIRECTORY ='val'

### Sample Values
TRAIN_DATASET_SAMPLE_COUNT=1000
TEST_DATASET_SAMPLE_COUNT=200
VALIDATION_DATASET_SAMPLE_COUNT=200

### Directories containing the images
SUB_DIRECTORIES = [TRAIN_DIRECTORY, TEST_DIRECTORY, VALIDATION_DIRECTORY]
DATA_DIRECTORIES = ['Actinic keratoses', 'Basal cell carcinoma', 'Benign keratosis-like lesions', 'Chickenpox', 'Cowpox', 'Dermatofibroma', 'Healthy', 'HFMD', 'Measles', 'Melanocytic nevi', 'Melanoma', 'Monkeypox', 'Squamous cell carcinoma', 'Vascular lesions']

### Data generation properties
ROTATION_RANGE = 40
WIDTH_SHIFT_RANGE = 0.2
HEIGHT_SHIFT_RANGE = 0.2
SHEAR_RANGE = 0.2
ZOOM_RANGE = 0.2
HORIZONTAL_FLIP = True
FILL_MODE = 'nearest'

### Model-related constants
IMG_HEIGHT = 224
IMG_WIDTH = 224
NUM_CLASSES = len(DATA_DIRECTORIES)
BATCH_SIZE = 32
EPOCHS = 20

## 🧮 Get Number of Images

Get the number of images in each directory and sub-directory to get an understanding of the dataset.

In [ ]:
## 🧮 Get Number of Images

### This method is used to get the number of images in each of the directories in our dataset

def count_images(dir_name):
    for dataset_type in SUB_DIRECTORIES:
        dir_type = os.path.join(dir_name, dataset_type)
        print(f"{dataset_type}")
        print("----------------------------")
        for category in os.listdir(dir_type):
            category_path = os.path.join(dir_type, category)
            if not os.path.isdir(category_path):
                continue
            images = [img for img in os.listdir(category_path) if img.endswith(('jpg', 'jpeg'))]
            print(f"Number of images in {category_path.split('/')[-1]}: {len(images)}")
        print("")

In [ ]:
### Execute the method

count_images(DATA_PATH)

## 🗜️ Image Augmentation for Categories with Low Image Count

Since the image count is low on the following we need to augment those images. Then, we can add the original images and a set of augmented images for our model creation.

***train*** Dataset (<1000)
- *Cowpox*
- *Actinic keratoses*
- *Measels*
- *Chickenpox*
- *Squamous cell carcinoma*
- *Dermatofibroma*
- *Vascular lesions*

***test*** Dataset (<200)
- *Cowpox*
- *Healthy*
- *Actinic keratoses*
- *Measels*
- *Chickenpox*
- *Squamous cell carcinoma*
- *Dermatofibroma*
- *Vascular lesions*

***val*** Dataset (<200)
- *Cowpox*
- *Healthy*
- *Actinic keratoses*
- *Measels*
- *Chickenpox*
- *Squamous cell carcinoma*
- *Dermatofibroma*
- *Vascular lesions*

### 💾 Define Data Generation Properties
To augment images we need to use something called `ImageDataGenerator`. To use that we need to define data-gen properties. This way more data is generated according to the parameters we define. The parameters we will be using for this are as follows.

- ↻ `rotation_range`
    - Specifies the range(in degrees) within which the image is randomly rotated.
    - For example, a value of `40` means the image can be rotated randomly by up to `40` degrees in either direction(clockwise or counterclockwise).<br><br>
    
- ↔ `width_shift_range`
    - Specifies the fraction of the total width of the image by which it can be randomly shifted horizontally.
    - For example, a value of `0.2` allows a horizontal shift of up to `20%` of the image's width.<br><br>
    
- ↕ `height_shift_range`
    - Specifies the fraction of the total height of the image by which it can be randomly shifted vertically.
    - For example, a value of `0.2` allows a vertical shift of up to `20%` of the image's height.<br><br>
  
- ➤ `shear_range`
    - Specifies the intensity of shear transformation as a shear angle in a counterclockwise direction in degrees.
    - A shear transformation distorts the image along an axis creating a parallelogram effect.<br><br>

- 🔎 `zoom_range`
   - Specifies the range for random zoom. It can zoom in or out within this range.
   - For example, a value of `0.2` means the image size can be varied randomly by up to `±20%`.<br><br>

- ― `horizontal_flip`
   - Specifies whether the image should be flipped horizontally.
   - If `True`, the image is randomly flipped along the horizontal axis, mirroring it.<br><br>

- 🚰 `fill_mode`
   - Specifies the strategy for filling in pixels that are introduced during transformations.
   - The value `nearest` means the nearest pixel values are used to fill these gaps.

In [ ]:
## 🗜️ Image Augmentation for Categories with Low Image Count

### Define directories that lack the images count
TRAIN_IMAGE_AUGEMENTATION_DIRS = ['Cowpox', 'Actinic keratoses', 'Measles', 'Chickenpox', 'Squamous cell carcinoma', 'Dermatofibroma', 'Vascular lesions']
TEST_IMAGE_AUGEMENTATION_DIRS = ['Cowpox', 'Healthy', 'Actinic keratoses', 'Measles', 'Chickenpox', 'Squamous cell carcinoma', 'Dermatofibroma', 'Vascular lesions']
VAL_IMAGE_AUGEMENTATION_DIRS = ['Cowpox', 'Healthy', 'Actinic keratoses', 'Measles', 'Chickenpox', 'Squamous cell carcinoma', 'Dermatofibroma', 'Vascular lesions']

### Create Image Generator
datagen = ImageDataGenerator(
    rotation_range=ROTATION_RANGE,
    width_shift_range=WIDTH_SHIFT_RANGE,
    height_shift_range=HEIGHT_SHIFT_RANGE,
    shear_range=SHEAR_RANGE,
    zoom_range=ZOOM_RANGE,
    horizontal_flip=HORIZONTAL_FLIP,
    fill_mode=FILL_MODE)

### Function to process and augment data
def augment_and_copy_data(dir_name):
    for dataset_type in SUB_DIRECTORIES:
        dir_type = os.path.join(dir_name, dataset_type)
        #### Dependeing on the dataset_type pick the categories with less number of images
        categories = []
        if dataset_type == TRAIN_DIRECTORY:
            categories = TRAIN_IMAGE_AUGEMENTATION_DIRS
        elif dataset_type == TEST_DIRECTORY:
            categories = TEST_IMAGE_AUGEMENTATION_DIRS
        else:
            categories = VAL_IMAGE_AUGEMENTATION_DIRS
        for category in categories:
            category_path = os.path.join(dir_type, category)
            #### Get augmented dataset path (e.g. ./data/augmented/train)
            augmented_path = os.path.join(AUGMENTED_PATH, dataset_type)
            #### Create the dataset type directory if it does not exist
            os.makedirs(augmented_path, exist_ok=True)
            #### Get augmented category path (e.g. ./data/augmented/train/Cowpox)
            augmented_category_path = os.path.join(augmented_path, category)
            #### Create a directory if it does not exist
            os.makedirs(augmented_category_path, exist_ok=True)

            #### Get all the files inside a category directory
            files = os.listdir(category_path)

            print(f'Started augmenting images in: {category_path}')

            #### Iterate files
            for file in files:
                if file.startswith('.'): ### Skip hidden files
                    continue
                #### Load images
                img_path = os.path.join(category_path, file)
                img = load_img(str(img_path))
                x = img_to_array(img) #### This is a NumPy array with shape (3, 150, 150)
                x = x.reshape((1,) + x.shape) #### This is a NumPy array with shape (1, 3, 150, 150)
                #### The .flow() command generates batches of randomly transformed images and saves the results in the ./data/augmented/<sub_directory>/<data_directory> along with the original image
                #### print(f'Started augmenting image: {img_path}')
                i = 0
                for batch in datagen.flow(x, batch_size=1, save_to_dir=augmented_category_path, save_prefix=file.split(".")[0], save_format='jpg'):
                    i += 1
                    if i > 10:
                        break #### Stop looping the generator infinitely

In [ ]:
### Execute the method

augment_and_copy_data(DATA_PATH)

## ⚖️ Create a Balanced Dataset

We need to create a balanced dataset to use the RNN models more effectively. The lowest number of images in category directories of the `train` dataset is 191. Therefore, we can create a balanced dataset containing 191 images from each category directory.

To do that we can create a method named `create_balanced_dataset` with three inputs, `source_dir`, `dest_dir`, and `num_images`. 

In [ ]:
## ⚖️ Create a Balanced Dataset
### This method is used to copy a certain number of images from every, train, test, and validation set.

def limit_data_from_source(source_dir, dest_dir, num_images=TRAIN_DATASET_SAMPLE_COUNT):
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)
    for dataset_type in SUB_DIRECTORIES:
        source_type_dir = os.path.join(source_dir, dataset_type)
        dest_type_dir = os.path.join(dest_dir, dataset_type)
        os.makedirs(dest_type_dir, exist_ok=True)
        for category in os.listdir(source_type_dir):
            category_path = os.path.join(source_type_dir, category)
            if not os.path.isdir(category_path):
                continue
            dest_category_path = os.path.join(dest_type_dir, category)
            os.makedirs(dest_category_path, exist_ok=True)

            images = [img for img in os.listdir(category_path) if img.endswith(('jpg', 'jpeg'))]
            print(f"Number of images in {dataset_type}/{category_path.split('/')[-1]}: {len(images)}")
            #### Select the number of images provided in num_images param or the minimum number of images
            if dataset_type == TRAIN_DIRECTORY:
                num_images = TRAIN_DATASET_SAMPLE_COUNT
            elif dataset_type == TEST_DIRECTORY:
                num_images = TEST_DATASET_SAMPLE_COUNT
            else:
                num_images = VALIDATION_DATASET_SAMPLE_COUNT
            selected_images = random.sample(images, min(num_images, len(images)))

            for image in selected_images:
                source_image_path = os.path.join(category_path, image)
                dest_image_path = os.path.join(dest_category_path, image)
                #### print(f'Copying {source_image_path} to {dest_image_path}')
                shutil.copy(source_image_path, dest_image_path)          

In [ ]:
limit_data_from_source(DATA_PATH, RNN_BALANCED_PATH)

## ➕ Add the Augmented Images to the Balanced Dataset to Balance the Number of Images

The next step is to identify the number of missing images from the balanced dataset and add the missing images from the augmented images.

In [ ]:
count_images(RNN_BALANCED_PATH)

In [ ]:
## Since there are missing images, we need to find out exactly how many images we are missing by creating dictiionaries for each of the test, train, and val directories.
## Function for getting missing image count for train, test, and val

def get_missing_images_count(dir_name):
    ## Create dictionaries for train, test, and validation
    train_dict = dict()
    test_dict = dict()
    val_dict = dict()
    for dataset_type in SUB_DIRECTORIES:
        dir_type = os.path.join(dir_name, dataset_type)
        for category in os.listdir(dir_type):
            category_path = os.path.join(dir_type, category)
            if not os.path.isdir(category_path):
                continue
            images = [img for img in os.listdir(category_path) if img.endswith(('jpg', 'jpeg'))]
            required_count = 0
            if dataset_type == TRAIN_DIRECTORY:
                image_count = min(len(images), TRAIN_DATASET_SAMPLE_COUNT)
                required_count = TRAIN_DATASET_SAMPLE_COUNT - image_count
                train_dict[category_path.split("/")[-1]] = required_count
            elif dataset_type == TEST_DIRECTORY:
                image_count = min(len(images), TEST_DATASET_SAMPLE_COUNT)
                required_count = TEST_DATASET_SAMPLE_COUNT - image_count
                test_dict[category_path.split("/")[-1]] = required_count
            else:
                image_count = min(len(images), VALIDATION_DATASET_SAMPLE_COUNT)
                required_count = VALIDATION_DATASET_SAMPLE_COUNT - image_count
                val_dict[category_path.split("/")[-1]] = required_count
    return train_dict, test_dict, val_dict

In [ ]:
## ➕ Add the Augmented Images to the Balanced Dataset to Balance the Number of Images

## Get the missing number of images
train_dict, test_dict, val_dict = get_missing_images_count(RNN_BALANCED_PATH)

## Print the missing values

def print_dict(dict_name, dataset_type):
    print(f"Missing values in {dataset_type}")
    print("----------------------------")
    for key, value in train_dict.items():
        print(f"{key} : {value}")

### print train_dict key, value pairs
print_dict(train_dict, TRAIN_DIRECTORY)

### print test_dict key, value pairs
print_dict(test_dict, TEST_DIRECTORY)

### print val_dict key, value pairs
print_dict(val_dict, VALIDATION_DIRECTORY)

In [ ]:
## Function to copy the missing number of images randomly from the augmented dataset

def copy_missing_images_from_augmented(src_path, dest_path):
    # Get the missing image count
    train_dict, test_dict, val_dict = get_missing_images_count(RNN_BALANCED_PATH)

    # Dictionary mapping dataset types to their corresponding missing counts
    missing_counts = {
        TRAIN_DIRECTORY: train_dict,
        TEST_DIRECTORY: test_dict,
        VALIDATION_DIRECTORY: val_dict,
    }

    for dataset_type in SUB_DIRECTORIES:
        src_dir_type = os.path.join(src_path, dataset_type)
        dest_dir_type = os.path.join(dest_path, dataset_type)

        # Pass the dictionary for the current dataset type
        copy_images_for_dataset_type(src_dir_type, dest_dir_type, missing_counts[dataset_type])

## Helper function to copy images for a specific dataset type

def copy_images_for_dataset_type(src_dir, dest_dir, missing_dict):
    for key, value in missing_dict.items():
        if value == 0:
            continue
        src_cat_dir = os.path.join(src_dir, key)
        dest_cat_dir = os.path.join(dest_dir, key)

        # Get all available images in the source category directory
        images = [img for img in os.listdir(src_cat_dir) if img.endswith(('jpg', 'jpeg'))]

        # Randomly select the missing number of images
        selected_images = random.sample(images, min(value, len(images)))

        # Copy the selected images to the destination directory
        for img in selected_images:
            shutil.copy(os.path.join(src_cat_dir, img), dest_cat_dir)

In [ ]:
## Copy the missing images from augmented images

copy_missing_images_from_augmented(AUGMENTED_PATH, RNN_BALANCED_PATH)

## Count the number of images again

train_dict, test_dict, val_dict = get_missing_images_count(RNN_BALANCED_PATH)

### print train_dict key, value pairs
print_dict(train_dict, TRAIN_DIRECTORY)

### print test_dict key, value pairs
print_dict(test_dict, TEST_DIRECTORY)

### print val_dict key, value pairs
print_dict(val_dict, VALIDATION_DIRECTORY)

## 🗃️ Creating File Data Frame for Train Dataset

Now, we have all the required images to make our RNN model.

Next, to make our RNN model, we need to create a data frame that will contain the paths and the names of the dermatological diseases. 

After creating the data frame, we can check whether there is a data imbalance with a pie chart.

In [ ]:
## 🗃️ Creating File Data Frame for Train Dataset

def load_images_to_df(base_path, dataset_type, sample_size):
    ### Load the base directory path
    path = os.path.join(base_path, dataset_path)
    image_dir = Path(path)

    ### Get file paths and assign labels
    file_paths = list(image_dir.glob(r'**/*.jpg'))
    labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], file_paths))

    file_paths = pd.Series(file_paths, name='Path').astype(str)
    labels = pd.Series(labels, name='Label')

    ### Concatenate file paths and labels
    image_df = pd.concat([file_paths, labels], axis=1)

    ### Get samples according to the sample size
    samples =[]
    for record in image_df['Label'].unique():
        samples.append(image_df[image_df['Label'] == record].sample(sample_size, random_state=42))
    image_df = pd.concat(samples, axis=0).sample(frac=1.0, random_state=42).reset_index(drop=True)

    ### Return the data frame
    return image_df

In [ ]:
### Get the images of the train set
image_df_train = load_images_to_df(RNN_BALANCED_PATH, TRAIN_DIRECTORY, TRAIN_DATASET_SAMPLE_COUNT)

### Show output
image_df_train.head(10)

In [ ]:
## Show the number of images to show there are no biases in the image set

pie = image_df_train['Label'].value_counts()
pie.plot(kind='pie', autopct='%1.2f%%')
plt.ylabel('')
plt.show()

## 🖥️ Display Images of the Dataset

Since we see that there is no data imbalance, the next step is to see whether our data has been loaded correctly or not in our data frame.

We can check that by plotting a sample of our images. To do that, we can use the `matplotlib` library.

In [ ]:
## 🖥️ Display Images of the Dataset

### Define figure and axes
fig, axes = plt.subplots(nrows=6, ncols=5, figsize=(15,8), subplot_kw={'xticks':[], 'yticks':[]})

### Display 30 images of the dataset
for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(image_df_train.Path[i], -1))
    ax.set_title(image_df_train.Label[i])
plt.tight_layout()
plt.show()

## 🗃️ Load the Test and Validation Data Frames

Next, load the test and validation data frames so that we can use them to create the models.

In [ ]:
## 🗃️ Load the Test and Validation Data Frames

image_df_test = load_images_to_df(RNN_BALANCED_PATH, TEST_DIRECTORY, TEST_DATASET_SAMPLE_COUNT)
image_df_test.head(10)

In [ ]:
image_df_val = load_images_to_df(RNN_BALANCED_PATH, VALIDATION_DIRECTORY, VALIDATION_DATASET_SAMPLE_COUNT)
image_df_val.head(10)

In [ ]:
def preprocess_images(df, img_height=IMG_HEIGHT, img_width=IMG_WIDTH):
    images = []
    labels = []
    for index, row in df.iterrows():
        img = tf.keras.utils.load_img(row['Path'], target_size=(img_height, img_width))
        img_array = tf.keras.utils.img_to_array(img)/255.0 # Normalize
        img_array = img_array.reshape(img_height, -1) # Flatten into rows(sequential data)
        images.append(img_array)
        labels.append(row['Label'])
    return np.array(images), np.array(labels)

In [ ]:
# Load training data
train_images, train_labels = preprocess_images(image_df_train)
val_images, val_labels = preprocess_images(image_df_val)
test_images, test_labels = preprocess_images(image_df_test)

In [ ]:
# Encode labels
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
val_labels = label_encoder.transform(test_labels)
test_labels = label_encoder.transform(test_labels)

In [ ]:
# Convert labels to one-hot encoding
train_labels = to_categorical(train_labels, NUM_CLASSES)
val_labels = to_categorical(val_labels, NUM_CLASSES)
test_labels = to_categorical(test_labels, NUM_CLASSES)

In [ ]:
def build_model_lstm(hp):
    model = Sequential()

    # First LSTM layer with tunable number of units
    model.add(LSTM(units=hp.Int('units_lstm_1', min_value=64, max_value=256, step=64),
    input_shape=(IMG_HEIGHT, IMG_WIDTH *3),
    return_sequences=True))

    # Second LSTM layer with tunable number of units
    model.add(LSTM(units=hp.Int('units_lstm_2', min_value=32, max_value=128, step=32)))

    # Dropout layer with tunable rate
    moel.add(Dropout(hp.Float('dropout_rate', min_value=0.2, max_value=0.5, step=0.1)))

    # Dense layer with tunable number of units
    model.add(Dense(units=hp.Int('units_dense', min_value=16, max_value=64, step=16), activation='relu'))

    # Output layer
    model.add(Dense(NUM_CLASSES, activation='softmax'))

    model.compile(
        optimizer=tf.keras.optimizers.legacy.Adam(
            learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
        ),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [ ]:
# Initialize the tuner
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=20,
    factor=3,
    directory='rnn_tuning',
    project_name='image_classification'
)

In [ ]:
# Define an early stopping callback
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
# Run the hyperparameter search
tuner.search(
    train_images, train_labels,
    validation_data=(val_images, val_labels),
    epochs=20,
    callbacks=[stop_early],
    batch_size=BATCH_SIZE
)

In [ ]:
# Get the best hyperparameters and model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"""
Best hyperparameters:
- Units in LSTM Layer 1: {best_hps.get('units_lstm_1')}
- Units in LSTM Layer 2: {best_hps.get('units_lstm_2')}
- Dropout Rate: {best_hps.get('dropout_rate')}
- Units in Dense Layer: {best_hps.get('units_dense')}
- Learning Rate: {best_hps.get('learning_rate')}
""")

In [ ]:
# Build the best model and retrain
best_model = tuner.hypermodel.build(best_hps)

In [ ]:
history = best_model.fit(train_images, train_labels, validation_data=(val_images, val_labels),
                    batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=[stop_early])

In [ ]:
# Evaluate the model
test_loss, test_accuracy = best_model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_accuracy:.2f}")

In [ ]:
# Classification Report
predictions = best_model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(test_labels, axis=1)
print(classification_report(true_classes, predicted_classes, target_names=label_encoder.classes_))

In [ ]:
# Accuracy Plot
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
# Loss Plot
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

In [ ]:
def build_gru_model(hp):
    model = Sequential()

    ### First GRU layer with tunable number of units
    model.add(GRU(units=hp.Int('units_gru_1', min_value=64, max_value=256, step=64), input_shape=(IMG_HEIGHT, IMG_WIDTH * 3), return_sequences=True))

    ### Second GRU layer with tunable number of units
    model.add(GRU(units=hp.Int('units_gru_2', min_value=32, max_value=128, step=32)))

    ### Dropout layer with tunable rate
    model.add(Dropout(hp.Float('dropout_rate', min_value=0.2, max_value=0.5, step=0.1)))

    ### Dense layer with tunable number of units
    model.add(Dense(units=hp.Int('units_dense', min_value=16, max_value=64, step=16), activation='relu'))

    ### Output layer
    model.add(Dense(NUM_CLASSES, activation='softmax'))

    ### Compile the model with tunable learning rate
    model.compile(optimizer = tf.keras.optimizers.legacy.Adam(
        learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    ))

    ### Return the model
    return model

In [ ]:
### Initialize the tuner for GRU
tuner = kt.Hyperband(
    build_gru_model,
    objectives='val_accuracy',
    max_epochs=20,
    factor=3,
    directory='gru_tuning',
    project_name='image_classification_gru'
)

In [ ]:
### Early stopping to avoid overfitting
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
### Run the hyperparameter search
tuner.search(
    train_images, train_labels,
    validation_data=(val_images, val_labels),
    epochs=20,
    callbacks=[stop_early],
    batch_size=BATCH_SIZE
)

In [ ]:
# Retrieve the best hyperparameters and model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"""
Best hyperparameters:
- Units in GRU Layer 1: {best_hps.get('units_gru_1')}
- Units in GRU Layer 2: {best_hps.get('units_gru_2')}
- Dropout Rate: {best_hps.get('dropout_rate')}
- Units in Dense Layer: {best_hps.get('units_dense')}
- Learning Rate: {best_hps.get('learning_rate')}
""")

In [ ]:
# Build the best GRU model and retrain it
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(
    train_images, train_labels,
    validation_data=(val_images, val_labels),
    epochs=20,
    batch_size=BATCH_SIZE,
    callbacks=[stop_early]
)

In [ ]:
# Evaluate the best GRU model
test_loss, test_accuracy = best_model.evaluate(test_images, test_labels)
print(f"Test Accuracy with Best Hyperparameters: {test_accuracy:.2f}")

In [ ]:
# Generate classification report
predictions = best_model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(test_labels, axis=1)
print(classification_report(true_classes, predicted_classes, target_names=label_encoder.classes_))

In [ ]:
# Accuracy plot
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Model Accuracy (GRU)')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
# Loss plot
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Model Loss (GRU)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()